# Effect of a carbon tax for planetery boundaries

## Model

This notebook provides a numerical simulation of the effect of a potential carbon tax on variables which have been identified as 
key drivers of the processes underlying the planetary boundaries (PB) framework. The model, upon which we base our analysis, is set up along the lines of a standard economic textbook model of decentralized competitive economy where firms and consumers have preferences over outcomes and each solve maximization problems by engaging in trade with each other. The sectors (or economic actors) we have included are those that we have identified as being central drivers of the processes underlying the PB's these include e.g. the production of fossil fuel, fertilizer, agriculture, timber, water and fisheries.  To study the implications of a carbon tax within this setting we have proceeded along the following lines:
1. Characterize the problem each economic actor is trying to solve given their preferences, technology and constraints.
2. Derive the corresponding first order conditions for each actor.
3. From the first order conditions characterize the equilibrium allocation only in terms of the model variables (i.e. eliminate prices) 
4. Derive the comparitive statics of the effect of a carbon tax i.e. differentiate the equilibrium allocation fully with respect to the carbon tax.

The resulting equilibrium outcome thus indirectly reveals how each variable in the model responds to a carbon tax. The model description and underlying 
the analytical derivations can be found at https://www.overleaf.com/6140270ztmfxn#/20533670/. Hint: in the comments below the equation numbers corresponding to this document are referenced when available. 

## Numerical solution method

The numerical solution is found as the solution to a system of 19 equations in 19 unknowns. This details of this equation system and how it was derived is found in the document referenced above (see equations 72-90). The solution method adopted below proceeds as follows:
1. From the equation system (72-90) all terms containing the carbon tax $\tau_E$ are moved to the left and all other terms to the right.
2. We then proceeded by rewriting the system in matrix/vector form where the rows correspond to the equations and columns are variables (e.g. the first row in the matrix holds the parameters of equation (72), the second row equation (73), etc.). The system can thus be written as $\mathbf{\tau} = \mathbf{A y}$ were $\mathbf{\tau}$ is a vector of containing the terms with the carbon tax, $\mathbf{y}$ are 19x1 vectors of outcome variables and $\mathbf{A}$ is a 19x19 matrix of parameters.
3. The solution to the system is thus found by inverting the matrix $\mathbf{A}$ and multiplying each side by this inverse i.e. $\mathbf{y} = \mathbf{A^{-1} \tau}$

## Code

The model can be simulated by highlighting each code cell below and clicking the "run button" above for each cell  
in consecutive order.

### Import the necessary computational modules

In [1]:
import numpy as np
import pandas as pd

### Specify default parameter estimates

In [17]:
# eos -> elasticity of substitution
sigma_U = 0.5    # eos: food and non food (utility)
sigma_F = 3.0    # eos: food and marine and food (nested utility)
sigma_nF = 2.0   # eos: timber, manufacturing and timber products (nested utility)
sigma_A = 2.0    # eos: land and non-land (agric. prod)
sigma_P = 0.5    # eos: phosphor and fossil fuels (fertil. prod)
sigma_nLA = 0.5  # eos: phosphor, energy and water (nested agric. prod)
sigma_Eps = 2.0  # eos: fossil fuel, renewables and biofuel (energy prod)

# supply elasticities (Lambda)
Lambda_R = 1.0        # supply elast. renewables
Lambda_E = 0.5        # supply elast. fossil fuel
Lambda_W = 1.0        # supply elast. water
Lambda_Fi = 1.0       # supply elast. fish
Lambda_Pho = 1.0      # supply elast. phosphate

# quantity shares (e.g. Q_LA = LA/L)
Q_LA = 0.6         # share of land used for agriculture
Q_LT = 0.3         # share of land used for timber
Q_EEps = 0.95      # share of fossil fuel used for energy prod.
Q_AB = 0.1         # share of agri. prod. used for biofuels prod.
Q_EpsA = 0.1       # share of energy used for agri. prod.

# factor shares (Gamma)
GammaU_F = 0.2             # factor share food (utility)
GammaF_Fi = 0.2            # factor share fish (food)
GammanF_Y = 0.8            # factor share manufacturing (non-food)
GammanF_LU = 0.1           # factor share unused land (non-food)
GammaA_LA = 0.4            # factor share land (agric. prod)
GammanLA_P = 0.2           # factor share phospor (non-land)
GammanLA_W = 0.2           # factor share water (non-land)
GammaP_EP = 0.2            # factor share fossil fuel (fertilizer prod.)
GammaEps_AB = 0.1          # factor share biofuels (energy services prod.)
GammaEps_EEps = 0.75       # factor share fossil fuel (energy services prod.)
GammaT_LT = 0.8            # factor share land (timber prod.)
GammaY_EpsY = 0.05         # factor share energy (manufacturing.)

# elasticity of conversion costs : V_T = (L_T/C_T)*(dC_T/dL_T)
V_T = 1
V_A = 1

# Carbon tax
tau_E = 0.1

#### Optional code: replaces default parameters above with params from Google sheets

Run cell below if you wish to replace default parameters above with parameters from a specific column in
https://docs.google.com/spreadsheets/d/1GW24m1qCDB-K6S07dpOwoK7CaeYObe6xQLGVh7BPEQY/edit?usp=sharing. In the Google sheets you can add a new column with parameter estimates. To use them in the model simulation you first replace 'xxx' in the params_df[['xxx']] below with the column name you wish to use and then run the cell.  

In [44]:
params_df = pd.read_csv('https://docs.google.com/spreadsheets/d/'+
                   '1GW24m1qCDB-K6S07dpOwoK7CaeYObe6xQLGVh7BPEQY/export?format=csv', index_col=0)

# choose Google sheet column
params_df = params_df[['version1']]

# return column params as local variables
params = {}
for index, row in params_df.iterrows():
    params[index] = row[0]
locals().update(params)

### Model parameters which can be derived from the specified values above

In [19]:
# quantity shares (e.g. Q_LA = LA/L)
Q_LU = 1-Q_LT-Q_LA # share of land used for recreation
Q_EP = 1-Q_EEps    # share of fossil fuel used for fertilizer prod.
Q_AF = 1-Q_AB      # share of agri. prod. used for food prod.
Q_EpsY = 1-Q_EpsA  # share of energy used for final goods prod.

# factor shares (Gamma)
GammaU_nF = 1-GammaU_F     # factor share non-food (utility)
GammaF_AF = 1-GammaF_Fi    # factor share agricultural (food)
GammanF_T = 1-GammanF_LU-GammanF_Y     # factor share timber (non-food)
GammaA_nLA = 1-GammaA_LA   # factor share non-land (agric. prod)
GammanLA_EpsA = 1-GammanLA_W-GammanLA_P     # factor share water (non-land)
GammaP_Pho = 1-GammaP_EP   # factor share phosphate (fertilizer prod.)
GammaEps_R = 1-GammaEps_AB-GammaEps_EEps  # factor share renewables (energy services prod.)
GammaT_LTLT = GammaT_LT-1  
GammaY_EpsYEpsY = GammaY_EpsY-1

GammaA_P = GammaA_nLA*GammanLA_P
GammaA_W = GammaA_nLA*GammanLA_W 
GammaA_EpsA = GammaA_nLA*GammanLA_EpsA 
GammaU_AF = GammaU_F*GammaF_AF
GammaU_Fi = GammaU_F*GammaF_Fi
GammaU_Y= GammaU_F*GammanF_Y
GammaU_LU= GammaU_F*GammanF_LU
GammaU_T= GammaU_F*GammanF_T

# cross-elasticities (L_A, (W, P, Eps)
GammaA_WP = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_P
GammaA_WEpsA = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_EpsA
GammaA_WW = ((GammaA_nLA-1)/sigma_A+1/sigma_nLA)*GammanLA_W-1/sigma_nLA
GammaA_WLA = GammaA_LA/sigma_A

GammaU_FiY = GammaU_Y/sigma_U
GammaU_FiAF = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_AF
GammaU_FiFi = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_Fi - 1/sigma_F
GammaU_FiLU = GammaU_LU/sigma_U
GammaU_FiT = GammaU_T/sigma_U

GammaU_YY = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_Y - 1/sigma_nF
GammaU_YAF = GammaU_AF/sigma_U
GammaU_YFi = GammaU_Fi/sigma_U
GammaU_YLU = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_LU
GammaU_YT  = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_T

GammaU_AFY  = GammaU_Y/sigma_U
GammaU_AFAF = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_AF - 1/sigma_F
GammaU_AFFi = ((GammaU_F-1)/sigma_U + 1/sigma_F)*GammaF_Fi
GammaU_AFLU = GammaU_LU/sigma_U
GammaU_AFT  = GammaU_T/sigma_U

GammaU_LUY = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_Y
GammaU_LUAF = GammaU_AF/sigma_U
GammaU_LUFi = GammaU_Fi/sigma_U
GammaU_LULU = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_LU - 1/sigma_nF
GammaU_LUT  = ((GammaU_nF-1)/sigma_U + 1/sigma_nF)*GammanF_T

GammaA_LAP = GammaA_P/sigma_A
GammaA_LAEpsA = GammaA_EpsA/sigma_A
GammaA_LAW = GammaA_W/sigma_A
GammaA_LALA = (GammaA_LA-1)/sigma_A

### Setup matrix of parameter estimates

In [20]:
# Variable order for columns of matrix an rows in the outcome vector: 
# L_A, L_T, L_U, E, E_eps, E_P, A, A_B, A_F, Eps, Eps_A, Eps_Y, P, W, Pho, R, Fi, T, Y

# Coefficient  Matrix (rows: equations, columns: variables)
# note: python vectors starts with index zero
coef_matrix = np.zeros((19, 19))

# Policy variable vector (carbon tax)
policy_vector = np.zeros((19, 1))

## Populate coefficient matrix and policy vector (note: document equation numbers specified in brackets)
# Fixed totat land: eq. [72]
coef_matrix[0,0] = Q_LA
coef_matrix[0,1] = Q_LT
coef_matrix[0,2] = Q_LU

# Market clearing constraint fossil fuel: eq. [73]
coef_matrix[1,3] = -1
coef_matrix[1,4] = Q_EEps
coef_matrix[1,5] = Q_EP

# Market clearing agric. output: eq. [74]
coef_matrix[2,6] = -1
coef_matrix[2,7] = Q_AB
coef_matrix[2,8] = Q_AF

# Market clearing energy. services: eq. [75]
coef_matrix[3,9] = -1
coef_matrix[3,10] = Q_EpsA
coef_matrix[3,11] = Q_EpsY

# Agricultural prod. : eq. [76]
coef_matrix[4,0] = GammaA_LA
coef_matrix[4,6] = -1
coef_matrix[4,10] = GammaA_EpsA
coef_matrix[4,12] = GammaA_P
coef_matrix[4,13] = GammaA_W

# Timber prod. : eq. [77]
coef_matrix[5,1] = GammaT_LT
coef_matrix[5,17] = -1

# Fertilizer prod. : eq. [78]
coef_matrix[6,12] = -1
coef_matrix[6,5] = GammaP_EP
coef_matrix[6,14] = GammaP_Pho

# Energy prod. : eq. [79]
coef_matrix[7,9] = -1
coef_matrix[7,7] = GammaEps_AB
coef_matrix[7,4] = GammaEps_EEps
coef_matrix[7,15] = GammaEps_R

# Final good prod. : eq. [80]
coef_matrix[8,18] = -1
coef_matrix[8,11] = GammaY_EpsY

# foc: eq. [81]
coef_matrix[9,4] = -1/sigma_Eps
coef_matrix[9,15] = 1/sigma_Eps + Lambda_R
coef_matrix[9,3] = -Lambda_E

policy_vector[9] = 1/(1+tau_E)

# foc eq. [82]
coef_matrix[10,7] = -1/sigma_Eps
coef_matrix[10,15] = 1/sigma_Eps + Lambda_R
coef_matrix[10,0] = GammaA_WLA
coef_matrix[10,13] = GammaA_WW-Lambda_W
coef_matrix[10,12] = GammaA_WP
coef_matrix[10,10] = GammaA_WEpsA

# foc: eq. [83]
coef_matrix[11,18] = GammaU_YY-GammaU_FiY
coef_matrix[11,8] = GammaU_YAF-GammaU_FiAF
coef_matrix[11,16] = GammaU_YFi-GammaU_FiFi + Lambda_Fi
coef_matrix[11,2] = GammaU_YLU-GammaU_FiLU
coef_matrix[11,17] = GammaU_YT-GammaU_FiT

coef_matrix[11,7] = GammaEps_AB/sigma_Eps
coef_matrix[11,4] = GammaEps_EEps/sigma_Eps
coef_matrix[11,15] = (GammaEps_R-1)/sigma_Eps - Lambda_R
coef_matrix[11,11] = GammaY_EpsYEpsY

# foc: eq. [84]
coef_matrix[12,8] = -1/sigma_F 
coef_matrix[12,16] = 1/sigma_F+Lambda_Fi 
coef_matrix[12,0] = GammaA_WLA
coef_matrix[12,12] = GammaA_WP
coef_matrix[12,13] = GammaA_WW-Lambda_W
coef_matrix[12,10] = GammaA_WEpsA

# foc: eq. [85]
coef_matrix[13,12] = -1/sigma_nLA 
coef_matrix[13,13] = 1/sigma_F+Lambda_W 
coef_matrix[13,14] = -Lambda_Pho+(GammaP_Pho-1)/sigma_P 
coef_matrix[13,5] = GammaP_EP/sigma_P 

# foc: eq. [86]
coef_matrix[14,10] = -1/sigma_nLA 
coef_matrix[14,13] = 1/sigma_nLA+Lambda_W 
coef_matrix[14,15] = -Lambda_R+(GammaEps_R-1)/sigma_Eps 
coef_matrix[14,7] = GammaEps_AB/sigma_Eps 
coef_matrix[14,4] = GammaEps_EEps/sigma_Eps 

# foc: eq. [87]
coef_matrix[15,14] = -1/sigma_P-Lambda_Pho 
coef_matrix[15,5] = 1/sigma_P 
coef_matrix[15,3] = Lambda_E 

policy_vector[15] = -1/(1+tau_E)

# foc: eq. [88]
coef_matrix[16,2] = -1/sigma_nF 
coef_matrix[16,17] = 1/sigma_nF 
coef_matrix[16,1] = V_T - GammaT_LTLT 

# foc: eq. [89]
coef_matrix[17,18] = GammaU_LUY-GammaU_AFY
coef_matrix[17,8]  = GammaU_LUAF-GammaU_AFAF
coef_matrix[17,16] = GammaU_LUFi-GammaU_AFFi
coef_matrix[17,2]  = GammaU_LULU-GammaU_AFLU
coef_matrix[17,17] = GammaU_LUT-GammaU_AFT

coef_matrix[17,0] = V_A-GammaA_LALA
coef_matrix[17,12] = -GammaA_LAP
coef_matrix[17,13] = -GammaA_LAW
coef_matrix[17,10] = -GammaA_LAEpsA

# foc: eq. [90]
coef_matrix[18,16] = -Lambda_Fi+GammaU_FiFi-GammaU_Fi
coef_matrix[18,18] = GammaU_FiY-GammaU_Y
coef_matrix[18,8] = GammaU_FiAF-GammaU_AF
coef_matrix[18,2] = GammaU_FiLU-GammaU_LU
coef_matrix[18,17] = GammaU_FiT-GammaU_T


### Solve for the equilibrium and return resulting variable vector

In [35]:
# Invert cooeficient matrix and multiply with policy vector
results = np.dot(np.linalg.inv(coef_matrix), policy_vector)

# prepare list of variable names and labels
var_list = [('$L_A$', '(land-share agriculture)'), ('$L_T$', '(land-share timber)'), ('$L_U$', '(land-share other)'), 
             ('$E\;\;$', '(fossil-fuel extracted)'), ('$E_{\mathcal{E}}$', ' (fossil-fuel use energy services)'), ('$E_P$', '(fossil-fuel use fertilizer prod.)'), 
             ('$A\;\;$', '(agriculture total production)'), ('$A_B$', '(agriculture prod. for biofuels)'), ('$A_F$', '(agriculture prod. for food)'), 
             ('$\mathcal{E}\;\;$', '(energy services)'), ('$\mathcal{E}_A$', '(energy for agriculture)'), ('$\mathcal{E}_Y$', '(energy services for final goods)'), 
             ('$P\;\;$', '(fertilizer production)'), ('$W\;$', '(water production)'), ('$\mathcal{P}\;\;$', '(phosphate extraction)'), 
             ('$R\;\;$', '(renewables production)'), ('$F\;\;$','(fisheries production)'), ('$T\;\;$', '(timber production)'), ('$Y\;\;$', '(final goods)')]
var_names, var_labels = zip(*var_list)
var_full_names = [x[0]+' '+'$\text{'+x[1]+'}$' for x in var_list]

# display results in table
df = pd.DataFrame(columns=['values'])
for id, var in enumerate(var_full_names):
    df.loc[var] = results[id]
df

,values
$L_A$ $\text{(land-share agriculture)}$,0.006255
$L_T$ $\text{(land-share timber)}$,-0.006053
$L_U$ $\text{(land-share other)}$,-0.019370
$E\;\;$ $\text{(fossil-fuel extracted)}$,-0.666263
$E_{\mathcal{E}}$ $\text{ (fossil-fuel use energy services)}$,-0.680505
$E_P$ $\text{(fossil-fuel use fertilizer prod.)}$,-0.395667
$A\;\;$ $\text{(agriculture total production)}$,-0.204951
$A_B$ $\text{(agriculture prod. for biofuels)}$,0.198385
$A_F$ $\text{(agriculture prod. for food)}$,-0.249767
$\mathcal{E}\;\;$ $\text{(energy services)}$,-0.466970
